## SAFE RESEARCH & REPORTING ASSISTANT
In this notebook we'll bring together our learnings on generative AI agents and tooling, and code an AI agent which incorporates a search engine for doing research using up to date information, two output guardrails for protecting against banned topics and toxicity, and a generative AI model to write a report.  
We'll use the smolagent framework to construct the AI agent.

To start, let's suppress any unecessary warnings.

In [1]:
from warnings import filterwarnings
filterwarnings('ignore')

Next, let's import the libraries we'll need. As before with llm-guard, we'll make sure we have the correct numpy version for llm-guard.

In [ ]:
%pip install -q smolagents numpy==1.25.1 llm-guard

We can now import the libraries we'll be using for our tools and agents.

In [3]:
import os
from google.colab import userdata
from smolagents import CodeAgent, HfApiModel, DuckDuckGoSearchTool, tool
from llm_guard import scan_output
from llm_guard.output_scanners import BanTopics

Now we can start to create our tools.  The first is our search tool which will use duckduckgo using the integrated duckduckgo search capability. We don't need to code anything for this as we can call the function directly from the smolagent import.

The next tool we'll define is our banned topics guardrail. Again we use the decorator and comments to start, and then define the _scanner_ handle with the topics we're interested in banning and the threshold for triggering the guardrail.  We then run the scanner and get back the results of the scanning. If the result is valid we'll return it, else we'll return a rejection message. We'll print out a message so we can check that the guardrail is called.

In [5]:
@tool
def out_guard(article: str)->str:
  """This is a guardrail to check the response for any banned topics. Banned topics are violence, religion or politics.
     It returns the value true if the response does not contain any of the banned topics.
     Args:
       article: The output from the tool that could potentially contain violence, politics, or religion
  """
  scanner=BanTopics(topics=['violence','religion','politics'],threshold=0.5)
  output=''
  result,is_valid,risk_score = scanner.scan(article,output)
  print("Response has been guarded and is",is_valid)
  return is_valid

We can now construct our agent.  We're specifying an agent which generates code and runs it to integrate the tooling. We're specifying the tools to use and directing the report writing to be done using one of the Huggingface Qwen AI coder models.

In [6]:
model = HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct",token=userdata.get('HF_TOKEN'))
agent = CodeAgent(tools=[DuckDuckGoSearchTool(),out_guard],model=model,max_steps=20)

We can now run the agent with a prompt.

In [ ]:
prompt="Search the web to get information for a research report on the latest developments in Quantum Computing. Use the out_guard tool to ensure they do not mention banned topics. Produce a report on the results."
agent.run(prompt)

If we check the run log of the agent, we can see where the tools were invoked, and we get the final result that we requested.